In [1]:
# code adapted from https://github.com/projectmesa/mesa/tree/main/examples/virus_on_network

# https://www.sciencedirect.com/science/article/pii/S0360835221003053

In [2]:
import math
from enum import Enum
import networkx as nx

from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
from mesa.space import NetworkGrid

import mesa
from mesa.visualization.modules import TextElement
from mesa.visualization.UserParam import UserSettableParameter
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.modules import ChartModule
from mesa.visualization.modules import NetworkModule
from mesa.visualization.modules import CanvasGrid

In [3]:
class State(Enum):
    SUSCEPTIBLE = 0
    INFECTED = 1
    RESISTANT = 2


def number_state(model, state):
    return sum([1 for a in model.grid.get_all_cell_contents() if a.state is state])


def number_infected(model):
    return number_state(model, State.INFECTED)


def number_susceptible(model):
    return number_state(model, State.SUSCEPTIBLE)


def number_resistant(model):
    return number_state(model, State.RESISTANT)


class VirusOnNetwork(Model):
    """A wine supply chain model with some number of agents and parameters"""

    def __init__(self, num_nodes=10, 
                 avg_node_degree=3, 
                 initial_outbreak_size=1, 
                 virus_spread_chance=0.4,
                 virus_check_frequency=0.4, 
                 recovery_chance=0.3, 
                 gain_resistance_chance=0.5):

        self.num_nodes = num_nodes
        prob = avg_node_degree / self.num_nodes
        self.G = nx.erdos_renyi_graph(n=self.num_nodes, p=prob)
        self.grid = NetworkGrid(self.G)
        self.schedule = RandomActivation(self)
        self.initial_outbreak_size = initial_outbreak_size if initial_outbreak_size <= num_nodes else num_nodes
        self.virus_spread_chance = virus_spread_chance
        self.virus_check_frequency = virus_check_frequency
        self.recovery_chance = recovery_chance
        self.gain_resistance_chance = gain_resistance_chance

        self.datacollector = DataCollector({"Infected": number_infected,
                                            "Susceptible": number_susceptible,
                                            "Resistant": number_resistant})

        # Create agents
        for i, node in enumerate(self.G.nodes()):
            a = VirusAgent(i, self, State.SUSCEPTIBLE, self.virus_spread_chance, self.virus_check_frequency,
                           self.recovery_chance, self.gain_resistance_chance)
            self.schedule.add(a)
            # Add the agent to the node
            self.grid.place_agent(a, node)

        # Infect some nodes
        
        # Increase neighbors by 1?
        infected_nodes = self.random.sample(self.G.nodes(), self.initial_outbreak_size)
        for a in self.grid.get_cell_list_contents(infected_nodes):
            a.state = State.INFECTED

        self.running = True
        self.datacollector.collect(self)

    def resistant_susceptible_ratio(self):
        try:
            return number_state(self, State.RESISTANT) / number_state(self, State.SUSCEPTIBLE)
        except ZeroDivisionError:
            return math.inf

    def step(self):
        self.schedule.step()
        
        # collect data
        self.datacollector.collect(self)

    def run_model(self, n):
        for i in range(n):
            self.step()


class VirusAgent(Agent):
    def __init__(self, unique_id, 
                 model, 
                 initial_state, 
                 virus_spread_chance, 
                 virus_check_frequency,
                 recovery_chance, 
                 gain_resistance_chance):
        super().__init__(unique_id, model)

        self.state = initial_state

        self.virus_spread_chance = virus_spread_chance
        self.virus_check_frequency = virus_check_frequency
        self.recovery_chance = recovery_chance
        self.gain_resistance_chance = gain_resistance_chance

    def try_to_infect_neighbors(self):
        neighbors_nodes = self.model.grid.get_neighbors(self.pos, include_center=False)
        susceptible_neighbors = [agent for agent in self.model.grid.get_cell_list_contents(neighbors_nodes) if
                                 agent.state is State.SUSCEPTIBLE]
        for a in susceptible_neighbors:
            if self.random.random() < self.virus_spread_chance:
                a.state = State.INFECTED

    def try_gain_resistance(self):
        if self.random.random() < self.gain_resistance_chance:
            self.state = State.RESISTANT

    def try_remove_infection(self):
        # Try to remove
        if self.random.random() < self.recovery_chance:
            # Success
            self.state = State.SUSCEPTIBLE
            self.try_gain_resistance()
        else:
            # Failed
            self.state = State.INFECTED

    def try_check_situation(self):
        if self.random.random() < self.virus_check_frequency:
            # Checking...
            if self.state is State.INFECTED:
                self.try_remove_infection()

    def step(self):
        if self.state is State.INFECTED:
            self.try_to_infect_neighbors()
        self.try_check_situation()

In [4]:
class WineSupplyChainAgent(Agent):
    def __init__(self, 
                 unique_id, 
                 model, 
                 initial_size, 
                 diesel_cost, 
                 diesel_biogas_mix,
                 transportation_shortage, 
                 drought_severity):
        super().__init__(unique_id, model)

        self.state = initial_state

        self.virus_spread_chance = virus_spread_chance
        self.virus_check_frequency = virus_check_frequency
        self.recovery_chance = recovery_chance
        self.gain_resistance_chance = gain_resistance_chance

    def try_to_infect_neighbors(self):
        neighbors_nodes = self.model.grid.get_neighbors(self.pos, include_center=False)
        susceptible_neighbors = [agent for agent in self.model.grid.get_cell_list_contents(neighbors_nodes) if
                                 agent.state is State.SUSCEPTIBLE]
        for a in susceptible_neighbors:
            if self.random.random() < self.virus_spread_chance:
                a.state = State.INFECTED

    def try_gain_resistance(self):
        if self.random.random() < self.gain_resistance_chance:
            self.state = State.RESISTANT

    def try_remove_infection(self):
        # Try to remove
        if self.random.random() < self.recovery_chance:
            # Success
            self.state = State.SUSCEPTIBLE
            self.try_gain_resistance()
        else:
            # Failed
            self.state = State.INFECTED

    def try_check_situation(self):
        if self.random.random() < self.virus_check_frequency:
            # Checking...
            if self.state is State.INFECTED:
                self.try_remove_infection()

    def step(self):
        if self.state is State.INFECTED:
            self.try_to_infect_neighbors()
        self.try_check_situation()

In [5]:
def network_portrayal(G):
    # The model ensures there is always 1 agent per node

    def node_color(agent):
        return {
            State.INFECTED: '#a24857',
            State.SUSCEPTIBLE: '#00FFFF'
        }.get(agent.state, '#a24857')

    def edge_color(agent1, agent2):
        if State.RESISTANT in (agent1.state, agent2.state):
            return '#FFFFFF'
        return '#e8e8e8' #e8e8e8

    def edge_width(agent1, agent2):
        if State.RESISTANT in (agent1.state, agent2.state):
            return 3
        return 2

    def get_agents(source, target):
        return G.nodes[source]['agent'][0], G.nodes[target]['agent'][0]
    
    def node_size(agent):
        return 10 #roind(size/State.RESISTANT, 0)

    portrayal = dict()
    
    portrayal['nodes'] = [{'size': node_size(agents[0]),
                           'color': node_color(agents[0]),
                           'tooltip': "id: {}<br>state: {}".format(agents[0].unique_id, agents[0].state.name),
                           }
                          for (_, agents) in G.nodes.data('agent')]

    portrayal['edges'] = [{'source': source,
                           'target': target,
                           'color': edge_color(*get_agents(source, target)),
                           'width': edge_width(*get_agents(source, target)),
                           }
                          for (source, target) in G.edges]

    return portrayal

network = NetworkModule(network_portrayal, 500, 500, library='d3')


In [6]:
model_params = {
    'num_nodes': UserSettableParameter('slider', 'Initial Number of agents', 60, 10, 100, 1,
                                       description='Choose how many agents to include in the model'),
    'avg_node_degree': UserSettableParameter('slider', 'Cost of Diesel', 7, 3, 16, 1,
                                             description='Avg Node Degree'),
#    'initial_outbreak_size': UserSettableParameter('slider', '# Using Diesel/Biogas Mixture', 1, 0, 1, 1,
#                                                   description='Initial Outbreak Size'),
#    'virus_spread_chance': UserSettableParameter('slider', 'Transportation Labor Shortage', 0.4, 0.0, 1.0, 0.1,
#                                                 description='Probability that susceptible neighbor will be infected'),
#    'virus_check_frequency': UserSettableParameter('slider', 'Agricultural Labor Shortage', 0.4, 0.0, 1.0, 0.1,
#                                                   description='Frequency the nodes check whether they are infected by '
#                                                               'a virus'),
#    'recovery_chance': UserSettableParameter('slider', 'Drought Severity Degree', 0.3, 0.0, 1.0, 0.1,
#                                             description='Probability that the virus will be removed'),
#    'gain_resistance_chance': UserSettableParameter('slider', '# Using Lighter Paper/Plastic Pkg', 0.5, 0.0, 1.0, 0.1,
#                                                    description='Probability that a recovered agent will become '
#                                                                'resistant to this virus in the future'),
}

server = ModularServer(VirusOnNetwork, [network], 'Post-Pandemic Wine Supply Chain Model', model_params)

server.port = 4456

server.launch()

Interface starting at http://127.0.0.1:4456


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}


Socket opened!
{"type":"reset"}
Socket opened!
{"type":"reset"}
{"type":"get_step","step":2019}
{"type":"get_step","step":2020}
{"type":"get_step","step":2021}
{"type":"get_step","step":2022}
{"type":"get_step","step":2023}
{"type":"get_step","step":2024}
{"type":"get_step","step":2025}
{"type":"get_step","step":2026}
{"type":"get_step","step":2027}
{"type":"get_step","step":2028}
{"type":"get_step","step":2029}
{"type":"get_step","step":2030}
{"type":"get_step","step":2031}
{"type":"get_step","step":2032}
{"type":"submit_params","param":"avg_node_degree","value":8}
{"type":"submit_params","param":"avg_node_degree","value":9}
{"type":"submit_params","param":"avg_node_degree","value":10}
{"type":"submit_params","param":"avg_node_degree","value":11}
{"type":"submit_params","param":"avg_node_degree","value":12}
{"type":"submit_params","param":"avg_node_degree","value":13}
{"type":"submit_params","param":"avg_node_degree","value":14}
{"type":"submit_params","param":"avg_node_degree","value

{"type":"submit_params","param":"avg_node_degree","value":11}
{"type":"submit_params","param":"avg_node_degree","value":10}
{"type":"submit_params","param":"avg_node_degree","value":9}
{"type":"submit_params","param":"avg_node_degree","value":8}
{"type":"submit_params","param":"avg_node_degree","value":7}
{"type":"submit_params","param":"avg_node_degree","value":6}
{"type":"submit_params","param":"avg_node_degree","value":4}
{"type":"submit_params","param":"avg_node_degree","value":3}
{"type":"submit_params","param":"avg_node_degree","value":16}
{"type":"submit_params","param":"avg_node_degree","value":15}
{"type":"submit_params","param":"avg_node_degree","value":14}
{"type":"submit_params","param":"avg_node_degree","value":13}
{"type":"submit_params","param":"avg_node_degree","value":12}
{"type":"submit_params","param":"avg_node_degree","value":11}
{"type":"submit_params","param":"avg_node_degree","value":10}
{"type":"submit_params","param":"avg_node_degree","value":9}
{"type":"submit